# Scalings CU Mixed Heating

In [1]:
%reset -f
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os 
import glob
from IPython.display import set_matplotlib_formats
from scipy.optimize import curve_fit
import re
import matplotlib.ticker as mticker

%config InlineBackend.figure_formats = ['svg']

In [2]:
plt.rcParams['savefig.dpi'] = 350
plt.rcParams['figure.figsize'] = [5, 5]
plt.rcParams['font.size'] = 14
plt.rcParams['lines.linewidth'] = 0.7
plt.rcParams['lines.markersize'] = 5.

In [3]:
main_path = 'C:\\Users\\super\\Dropbox\\Rice University\\501\\BL_Interactions_MH\\'
save_path = 'C:\\Users\\super\\Dropbox\\Rice University\\501\\BL_Interactions_Data\\'
importData = pd.read_csv(save_path+'\\data_MH.csv')

In [4]:
def tFunction(x,a,b):
    return x[0]**(3/4)*x[1]**(-1/4)*a + b

# def nuFunction(x,m,a,b):
#     return a*(x[1]-1000)**b + m*x[0]

def nuFunction(x,a,b):
    return a*(x[1]-1000)**b + 0.5*x[0]

def velFunction(x,nuF,tF):
    s = ((nuFunction(x,nuF[0],nuF[1],nuF[2]))/(tFunction(x,tF[0],tF[1])))**2
    return s

def getRsquared(popt,ydata,xdata,f):
    residuals = ydata- f(xdata, *popt)
    ss_res = np.sum(residuals**2)
    ss_tot = np.sum((ydata-np.mean(ydata))**2)
    r_squared = 1 - (ss_res / ss_tot)
    return r_squared

In [5]:
data_Ra = importData['Ra'].to_numpy()
data_H = importData['H'].to_numpy()
data_Ti = importData['Ti'].to_numpy()
data_nuT = importData['NuT'].to_numpy()
data_nub = importData['NuB'].to_numpy()
data_both = [data_H,data_Ra]
data_nu = np.vstack((data_nuT,data_nub+data_H)).flatten()-1
data_nu_both = np.hstack((data_both,data_both))

In [6]:
fitTi,_ = curve_fit(tFunction,data_both,data_Ti)
print(fitTi) # m*x + b
getRsquared(fitTi,data_Ti,data_both,tFunction)

[1.43720404 0.52055895]


0.9962467990260567

In [7]:
fitNu, _ = curve_fit(nuFunction,data_nu_both,data_nu)
print(fitNu) # m*H + a*(Ra-1000)^b
getRsquared(fitNu,data_nu,data_nu_both,nuFunction)

[0.41042747 0.27675347]


0.995104395383685

In [8]:
file = open(save_path+"\\MH_interiorTempScaling_Y(0)_&_slope.txt", "w") #y-intercept first, then slope
str = repr(fitTi[1])
file.write(str+'\n')
str = repr(fitTi[0])
file.write(str)
file.close()

# file = open(save_path+"\\MH_nusseltScaling_beta_slopeH_slopeRa.txt", "w") #beta exponent, then slope of H, then slope of Ra
# str = repr(fitNu[2])
# file.write(str+'\n')
# str = repr(fitNu[0])
# file.write(str+'\n')
# str = repr(fitNu[1])
# file.write(str)
# file.close()

file = open(save_path+"\\MH_nusseltScaling_beta_slopeH_slopeRa.txt", "w") #beta exponent, then slope of Ra
str = repr(fitNu[1])
file.write(str+'\n')
str = repr(fitNu[0])
file.write(str+'\n')
file.close()